In [1]:
import urllib3

import numpy as np

# Домашнее задание на повторение numpy
В задании все пункты необходимо выполнить с помощью numpy, без использования циклов/map/и тд! 

Задание выполняется __самостоятельно__. Категорически запрещено пользоваться чужими решениями заданий (даже смотреть их), в том числе, размещенными в открытых источниках (если таковые обнаружатся). При обнаружении списывания выставляется 0 за задание (в случае списывания у одногруппника/однокурскника - __ему/ей тоже__), а также подается докладная записка в учебный офис. Проверка на списывание проводится и автоматическими средствами, и вручную.

### История

В конце каждого семестра каждый семинарист сталкивается с необходимостью выставить студентам накопленную в семестре оценку. Часто семинаристы делают это с помощью Excel или GoogleSheets, но поверьте, с этим возникают проблемы. В задании вам надо помочь семинаристу некоторого курса выполнить эту непростую работу с помощью numpy :)

Выполните ячейку ниже, чтобы загрузить данные и визуально проверить корректность их считывания.

По строкам - студенты, по столбцам - их оценки. В курсе семинарист провел три проверочных (первые три колонки), выдал три домашних работы (следующие три колонки), кроме того, у студентов есть оценка за коллоквиум (последняя колонка).

*Для избежания ненужного в данном случае кэширования метод `loadtxt` был заменен на `fromstring`.*

In [2]:
http = urllib3.PoolManager()
r = http.request(
    'GET',
    'https://raw.githubusercontent.com/nadiinchi/iad2019/master/materials/data_hw1.txt'
)
lines = r.data.decode('utf8').split('\n')

marks = np.vstack(
    [
        np.fromstring(line, dtype=float, sep=' ')[np.newaxis, :]
            for line in lines if line.strip()
    ]
)

In [3]:
marks[:3]

array([[ 7. ,  8. , 10. ,  0. ,  0. ,  2.1,  1.5],
       [ 0. ,  6. ,  4.4,  4. ,  0. ,  5. ,  4.3],
       [ 3. ,  7. ,  8.7,  7.6,  7. ,  9.7, 10. ]])

### Число студентов (1 балл)
Сколько студентов в группе семинариста?

In [4]:
# Записать ответ в переменную num_studs
num_studs = marks.shape[0]

In [5]:
print(num_studs)

28


### Очень сложная проверочная, или просто добрый семинарист (1.5 балла)
Семинарист помнит, что первая проверочная получилась очень сложной, и максимальный балл в группе был далеко не 10. Поэтому он решил перенормировать баллы за эту проверочную на максимум по группе (поделить на максимум и умножить на 10) - чтобы у студента с максимальной оценкой получилось 10, а у остальных - просто оценка больше.

Каков максимальный балл за первую проверочную?

In [6]:
# Записать ответ в переменную max_mark
max_mark = marks[:, 0].max()

In [7]:
print(max_mark)

7.0


Разделите первый столбец на max_mark (для этого надо выделить столбец, поставить после соответствующего выражения знак = и присвоить новое значение столбца)

In [8]:
marks[:, 0] = np.round(10.0 * marks[:, 0] / max_mark, 1)

In [9]:
print(marks[:3])

[[10.   8.  10.   0.   0.   2.1  1.5]
 [ 0.   6.   4.4  4.   0.   5.   4.3]
 [ 4.3  7.   8.7  7.6  7.   9.7 10. ]]


### Подсчет накопленной оценки (1.5 балла)
Для вычисления накопленной надо посчитать средние оценки за проверочные для каждого студента и средние за домашние работы (может пригодиться np.mean). Накопленная оценка есть 0.3\*Проверочные + 0.4\*Домашние + 0.3\*Коллоквиум. Вычислите накопленные оценки и добавьте их в последний столбец матрицы (нужно добавить еще один новый столбец).

In [10]:
# Веса следует брать из переменной weights, а не вписывать вручную
weights = [0.3, 0.4, 0.3]

In [11]:
means = np.hstack(
    [
        marks[:, :3].mean(axis=1)[:, np.newaxis],
        marks[:, 3:-1].mean(axis=1)[:, np.newaxis],
        marks[:, -1][:, np.newaxis],
    ]
)

In [12]:
marks = np.hstack(
    [
        marks,
        means @ np.array(weights)[:, np.newaxis],
    ]
)

In [13]:
marks[:, -1] = np.round(marks[:, -1], 2)

In [14]:
print(marks[:3])

[[10.    8.   10.    0.    0.    2.1   1.5   3.53]
 [ 0.    6.    4.4   4.    0.    5.    4.3   3.53]
 [ 4.3   7.    8.7   7.6   7.    9.7  10.    8.24]]


### Студенты, которых, вероятно, отчислят (1.5 балла)

Сколько в группе студентов с накопленной оценкой 0? Вам может понадобиться тот факт, что применение np.sum к бинарному вектору дает число значений True в нем, например:

In [15]:
np.sum([True, False, True]) # 2 значения True

2

Также обратим внимание на операцию поэлементного сравнения, возвращающую массив из True и False:

In [16]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 3])
print(a == b)
print(a > b)

[False False  True]
[False False False]


In [17]:
print(np.sum(marks[:, -1] == 0.0))

3


### Студенты, которые получат автоматы (1.5 балла)
Автомат получат студенты, у которых накопленная оценка больше 7. Добавьте еще один столбец - в нем стоит 1, если студент получает автомат, и 0 иначе.

In [18]:
marks = np.hstack(
    [
        marks,
        (marks[:, -1] > 7.0)[:, np.newaxis].astype(marks.dtype),
    ]
)

In [19]:
print(marks[:3])

[[10.    8.   10.    0.    0.    2.1   1.5   3.53  0.  ]
 [ 0.    6.    4.4   4.    0.    5.    4.3   3.53  0.  ]
 [ 4.3   7.    8.7   7.6   7.    9.7  10.    8.24  1.  ]]


### Студенты, которые будут освобождены от задач на экзамене (1.5 балла)
Семинарист решил, что первая половина группы в рейтинге по накопленной оценке будет освобождена от решения задач на экзамене. Выделите этих студентов в отдельный массив. Для этого надо отсортировать строки массива (np.argsort) и взять первую половину.

In [20]:
idx = np.argsort(marks[:, -2])[::-1]

In [21]:
selected = marks[idx[:idx.size >> 1], :]

In [22]:
print(selected)

[[10.   10.   10.    9.7  10.   10.   10.    9.96  1.  ]
 [ 9.3   8.7   9.5  10.5   9.8  10.   10.    9.79  1.  ]
 [10.    9.7  10.    8.1  10.   10.    9.5   9.57  1.  ]
 [ 7.1  10.   10.    9.9  10.5  10.    9.    9.46  1.  ]
 [ 5.7  10.   10.    9.9   9.7  10.    9.    9.22  1.  ]
 [10.   10.   10.    9.    9.5  10.    8.    9.2   1.  ]
 [ 4.3   8.7  10.    9.4  10.7  10.    9.    9.01  1.  ]
 [ 8.6  10.    9.7   8.7   7.    8.5   9.7   8.97  1.  ]
 [ 5.    9.3  10.    9.9  10.   10.    8.    8.82  1.  ]
 [ 8.6   5.   10.    9.7  10.   10.    8.    8.72  1.  ]
 [ 7.1  10.    9.    7.    7.9  10.    9.    8.63  1.  ]
 [ 0.    8.7   8.    9.7  10.    9.5  10.    8.56  1.  ]
 [ 8.6  10.   10.    9.7   7.   10.    7.    8.52  1.  ]
 [ 4.3   7.    8.7   7.6   7.    9.7  10.    8.24  1.  ]]


### Сложность заданий (1.5 балла)
Наконец, семинарист хочет оценить сложность заданий, проверочных и коллоквиума по своей особой системе. Он присвоил каждому студенту вес (веса суммируются в 1) и хочет посчитать для каждого задания, проверочной и коллоквиума среднюю оценку с весами.
Пример, что имеется в виду: если есть 4 студента с весами [0.1, 0.2, 0.3, 0.4] и оценками [3, 7, 9, 10], то средняя оценка за задание с весами получится 

In [23]:
0.1 * 3 + 0.2 * 7 + 0.3 * 9 + 0.4 * 10 

8.4

Веса:

In [24]:
weights = np.array([1.]*7+[2.]*4+[3.]*7+[4.]*(marks.shape[0]-18))
# разделите веса на их сумму, чтобы они суммировались к 1
weights /= np.sum(weights)

assert np.isclose(weights.sum(), 1.0)

Посчитайте указанным образом сложность каждого "столбца".

In [25]:
complexity = weights[np.newaxis, :] @ marks[:, :-2]

In [26]:
print(np.round(complexity.ravel(), 2))

[4.9  7.12 7.86 6.16 4.83 5.88 4.87]
